# **Import Libraries**

In [1]:
# Import Libraries
import csv
import re
import string
from io import StringIO

import pandas as pd
import requests

import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, Bidirectional, LSTM, GRU, Embedding
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

import warnings
warnings.filterwarnings("ignore")

nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\nurri\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\nurri\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

# **Loading Dataset**

In [2]:
# Read the CSV file into a DataFrame
app_reviews_df = pd.read_csv('reviews_tiktok.csv')

In [3]:
# Display the first five rows of the DataFrame
app_reviews_df.head()

,Review
0,"untuk apk ini Bagusss banget!! aku suka, tapi ..."
1,tiktok sekarang aplikasi nya berat banget🥲🥲......
2,awalnya aku kira hpku yg rusak soalnya setiap ...
3,"tiktok, sekarang banyak updatean tapi malah ta..."
4,"sebenarnya aplikasi nya sudah bagus, hanya saj..."


In [4]:
# Show summary information of the raw reviews DataFrame
app_reviews_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58500 entries, 0 to 58499
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Review  58500 non-null  object
dtypes: object(1)
memory usage: 457.2+ KB


In [5]:
# Remove duplicate
clean_df = app_reviews_df.drop_duplicates()

In [6]:
# Show summary information of the cleaned DataFrame
clean_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 58439 entries, 0 to 58499
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Review  58439 non-null  object
dtypes: object(1)
memory usage: 913.1+ KB


# **Preprocessing**

In [7]:
# Preprocess the text data
def cleaning_text(text):
    text = re.sub(r'@[A-Za-z0-9]+', '', text)
    text = re.sub(r'#[A-Za-z0-9]+', '', text)
    text = re.sub(r'RT[\s]', '', text)
    text = re.sub(r"http\S+", '', text)
    text = re.sub(r'[0-9]+', '', text)
    text = re.sub(r'[^\w\s]', '', text)
    text = text.replace('\n', ' ')
    text = text.translate(str.maketrans('', '', string.punctuation))
    text = text.strip()
    return text

def casefolding_text(text):
    return text.lower()

slangwords = {"@": "di", "abis": "habis", "wtb": "beli", "masi": "masih", "wts": "jual", "wtt": "tukar", "bgt": "banget", "maks": "maksimal", "plisss": "tolong", "bgttt": "banget", "indo": "indonesia", "bgtt": "banget", "ad": "ada", "rv": "redvelvet", "plis": "tolong", "pls": "tolong", "cr": "sumber", "cod": "bayar ditempat", "adlh": "adalah", "afaik": "as far as i know", "ahaha": "haha", "aj": "saja", "ajep-ajep": "dunia gemerlap", "ak": "saya", "akika": "aku", "akkoh": "aku", "akuwh": "aku", "alay": "norak", "alow": "halo", "ambilin": "ambilkan", "ancur": "hancur", "anjrit": "anjing", "anter": "antar", "ap2": "apa-apa", "apasih": "apa sih", "apes": "sial", "aps": "apa", "aq": "saya", "aquwh": "aku", "asbun": "asal bunyi", "aseekk": "asyik", "asekk": "asyik", "asem": "asam", "aspal": "asli tetapi palsu", "astul": "asal tulis", "ato": "atau", "au ah": "tidak mau tahu", "awak": "saya", "ay": "sayang", "ayank": "sayang", "b4": "sebelum", "bakalan": "akan", "bandes": "bantuan desa", "bangedh": "banget", "banpol": "bantuan polisi", "banpur": "bantuan tempur", "basbang": "basi", "bcanda": "bercanda", "bdg": "bandung", "begajulan": "nakal", "beliin": "belikan", "bencong": "banci", "bentar": "sebentar", "ber3": "bertiga", "beresin": "membereskan", "bete": "bosan", "beud": "banget", "bg": "abang", "bgmn": "bagaimana", "bgt": "banget", "bijimane": "bagaimana", "bintal": "bimbingan mental", "bkl": "akan", "bknnya": "bukannya", "blegug": "bodoh", "blh": "boleh", "bln": "bulan", "blum": "belum", "bnci": "benci", "bnran": "yang benar", "bodor": "lucu", "bokap": "ayah", "boker": "buang air besar", "bokis": "bohong", "boljug": "boleh juga", "bonek": "bocah nekat", "boyeh": "boleh", "br": "baru", "brg": "bareng", "bro": "saudara laki-laki", "bru": "baru", "bs": "bisa", "bsen": "bosan", "bt": "buat", "btw": "ngomong-ngomong", "buaya": "tidak setia", "bubbu": "tidur", "bubu": "tidur", "bumil": "ibu hamil", "bw": "bawa", "bwt": "buat", "byk": "banyak", "byrin": "bayarkan", "cabal": "sabar", "cadas": "keren", "calo": "makelar", "can": "belum", "capcus": "pergi", "caper": "cari perhatian", "ce": "cewek", "cekal": "cegah tangkal", "cemen": "penakut", "cengengesan": "tertawa", "cepet": "cepat", "cew": "cewek", "chuyunk": "sayang", "cimeng": "ganja", "cipika cipiki": "cium pipi kanan cium pipi kiri", "ciyh": "sih", "ckepp": "cakep", "ckp": "cakep", "cmiiw": "correct me if i'm wrong", "cmpur": "campur", "cong": "banci", "conlok": "cinta lokasi", "cowwyy": "maaf", "cp": "siapa", "cpe": "capek", "cppe": "capek", "cucok": "cocok", "cuex": "cuek", "cumi": "Cuma miscall", "cups": "culun", "curanmor": "pencurian kendaraan bermotor", "curcol": "curahan hati colongan", "cwek": "cewek", "cyin": "cinta", "d": "di", "dah": "deh", "dapet": "dapat", "de": "adik", "dek": "adik", "demen": "suka", "deyh": "deh", "dgn": "dengan", "diancurin": "dihancurkan", "dimaafin": "dimaafkan", "dimintak": "diminta", "disono": "di sana", "dket": "dekat", "dkk": "dan kawan-kawan", "dll": "dan lain-lain", "dlu": "dulu", "dngn": "dengan", "dodol": "bodoh", "doku": "uang", "dongs": "dong", "dpt": "dapat", "dri": "dari", "drmn": "darimana", "drtd": "dari tadi", "dst": "dan seterusnya", "dtg": "datang", "duh": "aduh", "duren": "durian", "ed": "edisi", "egp": "emang gue pikirin", "eke": "aku", "elu": "kamu", "emangnya": "memangnya", "emng": "memang", "endak": "tidak", "enggak": "tidak", "envy": "iri", "ex": "mantan", "fax": "facsimile", "fifo": "first in first out", "folbek": "follow back", "fyi": "sebagai informasi", "gaada": "tidak ada uang", "gag": "tidak", "gaje": "tidak jelas", "gak papa": "tidak apa-apa", "gan": "juragan", "gaptek": "gagap teknologi", "gatek": "gagap teknologi", "gawe": "kerja", "gbs": "tidak bisa", "gebetan": "orang yang disuka", "geje": "tidak jelas", "gepeng": "gelandangan dan pengemis", "ghiy": "lagi", "gile": "gila", "gimana": "bagaimana", "gino": "gigi nongol", "githu": "gitu", "gj": "tidak jelas", "gmana": "bagaimana", "gn": "begini", "goblok": "bodoh", "golput": "golongan putih", "gowes": "mengayuh sepeda", "gpny": "tidak punya", "gr": "gede rasa", "gretongan": "gratisan", "gtau": "tidak tahu", "gua": "saya", "guoblok": "goblok", "gw": "saya", "ha": "tertawa", "haha": "tertawa", "hallow": "halo", "hankam": "pertahanan dan keamanan", "hehe": "he", "helo": "halo", "hey": "hai", "hlm": "halaman", "hny": "hanya", "hoax": "isu bohong", "hr": "hari", "hrus": "harus", "hubdar": "perhubungan darat", "huff": "mengeluh", "hum": "rumah", "humz": "rumah", "ilang": "hilang", "ilfil": "tidak suka", "imho": "in my humble opinion", "imoetz": "imut", "item": "hitam", "itungan": "hitungan", "iye": "iya", "ja": "saja", "jadiin": "jadi", "jaim": "jaga image", "jayus": "tidak lucu", "jdi": "jadi", "jem": "jam", "jga": "juga", "jgnkan": "jangankan", "jir": "anjing", "jln": "jalan", "jomblo": "tidak punya pacar", "jubir": "juru bicara", "jutek": "galak", "k": "ke", "kab": "kabupaten", "kabor": "kabur", "kacrut": "kacau", "kadiv": "kepala divisi", "kagak": "tidak", "kalo": "kalau", "kampret": "sialan", "kamtibmas": "keamanan dan ketertiban masyarakat", "kamuwh": "kamu", "kanwil": "kantor wilayah", "karna": "karena", "kasubbag": "kepala subbagian", "katrok": "kampungan", "kayanya": "kayaknya", "kbr": "kabar", "kdu": "harus", "kec": "kecamatan", "kejurnas": "kejuaraan nasional", "kekeuh": "keras kepala", "kel": "kelurahan", "kemaren": "kemarin", "kepengen": "mau", "kepingin": "mau", "kepsek": "kepala sekolah", "kesbang": "kesatuan bangsa", "kesra": "kesejahteraan rakyat", "ketrima": "diterima", "kgiatan": "kegiatan", "kibul": "bohong", "kimpoi": "kawin", "kl": "kalau", "klianz": "kalian", "kloter": "kelompok terbang", "klw": "kalau", "km": "kamu", "kmps": "kampus", "kmrn": "kemarin", "knal": "kenal", "knp": "kenapa", "kodya": "kota madya", "komdis": "komisi disiplin", "komsov": "komunis sovyet", "kongkow": "kumpul bareng teman-teman", "kopdar": "kopi darat", "korup": "korupsi", "kpn": "kapan", "krenz": "keren", "krm": "kirim", "kt": "kita", "ktmu": "ketemu", "ktr": "kantor", "kuper": "kurang pergaulan", "kw": "imitasi", "kyk": "seperti", "la": "lah", "lam": "salam", "lamp": "lampiran", "lanud": "landasan udara", "latgab": "latihan gabungan", "lebay": "berlebihan", "leh": "boleh", "lelet": "lambat", "lemot": "lambat", "lgi": "lagi", "lgsg": "langsung", "liat": "lihat", "litbang": "penelitian dan pengembangan", "lmyn": "lumayan", "lo": "kamu", "loe": "kamu", "lola": "lambat berfikir", "louph": "cinta", "low": "kalau", "lp": "lupa", "luber": "langsung, umum, bebas, dan rahasia", "luchuw": "lucu", "lum": "belum", "luthu": "lucu", "lwn": "lawan", "maacih": "terima kasih", "mabal": "bolos", "macem": "macam", "macih": "masih", "maem": "makan", "magabut": "makan gaji buta", "maho": "homo", "mak jang": "kaget", "maksain": "memaksa", "malem": "malam", "mam": "makan", "maneh": "kamu", "maniez": "manis", "mao": "mau", "masukin": "masukkan", "melu": "ikut", "mepet": "dekat sekali", "mgu": "minggu", "migas": "minyak dan gas bumi", "mikol": "minuman beralkohol", "miras": "minuman keras", "mlah": "malah", "mngkn": "mungkin", "mo": "mau", "mokad": "mati", "moso": "masa", "mpe": "sampai", "msk": "masuk", "mslh": "masalah", "mt": "makan teman", "mubes": "musyawarah besar", "mulu": "melulu", "mumpung": "selagi", "munas": "musyawarah nasional", "muntaber": "muntah dan berak", "musti": "mesti", "muupz": "maaf", "mw": "now watching", "n": "dan", "nanam": "menanam", "nanya": "bertanya", "napa": "kenapa", "napi": "narapidana", "napza": "narkotika, alkohol, psikotropika, dan zat adiktif ", "narkoba": "narkotika, psikotropika, dan obat terlarang", "nasgor": "nasi goreng", "nda": "tidak", "ndiri": "sendiri", "ne": "ini", "nekolin": "neokolonialisme", "nembak": "menyatakan cinta", "ngabuburit": "menunggu berbuka puasa", "ngaku": "mengaku", "ngambil": "mengambil", "nganggur": "tidak punya pekerjaan", "ngapah": "kenapa", "ngaret": "terlambat", "ngasih": "memberikan", "ngebandel": "berbuat bandel", "ngegosip": "bergosip", "ngeklaim": "mengklaim", "ngeksis": "menjadi eksis", "ngeles": "berkilah", "ngelidur": "menggigau", "ngerampok": "merampok", "ngga": "tidak", "ngibul": "berbohong", "ngiler": "mau", "ngiri": "iri", "ngisiin": "mengisikan", "ngmng": "bicara", "ngomong": "bicara", "ngubek2": "mencari-cari", "ngurus": "mengurus", "nie": "ini", "nih": "ini", "niyh": "nih", "nmr": "nomor", "nntn": "nonton", "nobar": "nonton bareng", "np": "now playing", "ntar": "nanti", "ntn": "nonton", "numpuk": "bertumpuk", "nutupin": "menutupi", "nyari": "mencari", "nyekar": "menyekar", "nyicil": "mencicil", "nyoblos": "mencoblos", "nyokap": "ibu", "ogah": "tidak mau", "ol": "online", "ongkir": "ongkos kirim", "oot": "out of topic", "org2": "orang-orang", "ortu": "orang tua", "otda": "otonomi daerah", "otw": "on the way, sedang di jalan", "pacal": "pacar", "pake": "pakai", "pala": "kepala", "pansus": "panitia khusus", "parpol": "partai politik", "pasutri": "pasangan suami istri", "pd": "pada", "pede": "percaya diri", "pelatnas": "pemusatan latihan nasional", "pemda": "pemerintah daerah", "pemkot": "pemerintah kota", "pemred": "pemimpin redaksi", "penjas": "pendidikan jasmani", "perda": "peraturan daerah", "perhatiin": "perhatikan", "pesenan": "pesanan", "pgang": "pegang", "pi": "tapi", "pilkada": "pemilihan kepala daerah", "pisan": "sangat", "pk": "penjahat kelamin", "plg": "paling", "pmrnth": "pemerintah", "polantas": "polisi lalu lintas", "ponpes": "pondok pesantren", "pp": "pulang pergi", "prg": "pergi", "prnh": "pernah", "psen": "pesan", "pst": "pasti", "pswt": "pesawat", "pw": "posisi nyaman", "qmu": "kamu", "rakor": "rapat koordinasi", "ranmor": "kendaraan bermotor", "re": "reply", "ref": "referensi", "rehab": "rehabilitasi", "rempong": "sulit", "repp": "balas", "restik": "reserse narkotika", "rhs": "rahasia", "rmh": "rumah", "ru": "baru", "ruko": "rumah toko", "rusunawa": "rumah susun sewa", "ruz": "terus", "saia": "saya", "salting": "salah tingkah", "sampe": "sampai", "samsek": "sama sekali", "sapose": "siapa", "satpam": "satuan pengamanan", "sbb": "sebagai berikut", "sbh": "sebuah", "sbnrny": "sebenarnya", "scr": "secara", "sdgkn": "sedangkan", "sdkt": "sedikit", "se7": "setuju", "sebelas dua belas": "mirip", "sembako": "sembilan bahan pokok", "sempet": "sempat", "sendratari": "seni drama tari", "sgt": "sangat", "shg": "sehingga", "siech": "sih", "sikon": "situasi dan kondisi", "sinetron": "sinema elektronik", "siramin": "siramkan", "sj": "saja", "skalian": "sekalian", "sklh": "sekolah", "skt": "sakit", "slesai": "selesai", "sll": "selalu", "slma": "selama", "slsai": "selesai", "smpt": "sempat", "smw": "semua", "sndiri": "sendiri", "soljum": "sholat jumat", "songong": "sombong", "sory": "maaf", "sosek": "sosial-ekonomi", "sotoy": "sok tahu", "spa": "siapa", "sppa": "siapa", "spt": "seperti", "srtfkt": "sertifikat", "stiap": "setiap", "stlh": "setelah", "suk": "masuk", "sumpek": "sempit", "syg": "sayang", "t4": "tempat", "tajir": "kaya", "tau": "tahu", "taw": "tahu", "td": "tadi", "tdk": "tidak", "teh": "kakak perempuan", "telat": "terlambat", "telmi": "telat berpikir", "temen": "teman", "tengil": "menyebalkan", "tepar": "terkapar", "tggu": "tunggu", "tgu": "tunggu", "thankz": "terima kasih", "thn": "tahun", "tilang": "bukti pelanggaran", "tipiwan": "TvOne", "tks": "terima kasih", "tlp": "telepon", "tls": "tulis", "tmbah": "tambah", "tmen2": "teman-teman", "tmpah": "tumpah", "tmpt": "tempat", "tngu": "tunggu", "tnyta": "ternyata", "tokai": "tai", "toserba": "toko serba ada", "tpi": "tapi", "trdhulu": "terdahulu", "trima": "terima kasih", "trm": "terima", "trs": "terus", "trutama": "terutama", "ts": "penulis", "tst": "tahu sama tahu", "ttg": "tentang", "tuch": "tuh", "tuir": "tua", "tw": "tahu", "u": "kamu", "ud": "sudah", "udah": "sudah", "ujg": "ujung", "ul": "ulangan", "unyu": "lucu", "uplot": "unggah", "urang": "saya", "usah": "perlu", "utk": "untuk", "valas": "valuta asing", "w/": "dengan", "wadir": "wakil direktur", "wamil": "wajib militer", "warkop": "warung kopi", "warteg": "warung tegal", "wat": "buat", "wkt": "waktu", "wtf": "what the fuck", "xixixi": "tertawa", "ya": "iya", "yap": "iya", "yaudah": "ya sudah", "yawdah": "ya sudah", "yg": "yang", "yl": "yang lain", "yo": "iya", "yowes": "ya sudah", "yup": "iya", "7an": "tujuan", "ababil": "abg labil", "acc": "accord", "adlah": "adalah", "adoh": "aduh", "aha": "tertawa", "aing": "saya", "aja": "saja", "ajj": "saja", "aka": "dikenal juga sebagai", "akko": "aku", "akku": "aku", "akyu": "aku", "aljasa": "asal jadi saja", "ama": "sama", "ambl": "ambil", "anjir": "anjing", "ank": "anak", "ap": "apa", "apaan": "apa", "ape": "apa", "aplot": "unggah", "apva": "apa", "aqu": "aku", "asap": "sesegera mungkin", "aseek": "asyik", "asek": "asyik", "aseknya": "asyiknya", "asoy": "asyik", "astrojim": "astagfirullahaladzim", "ath": "kalau begitu", "atuh": "kalau begitu", "ava": "avatar", "aws": "awas", "ayang": "sayang", "ayok": "ayo", "bacot": "banyak bicara", "bales": "balas", "bangdes": "pembangunan desa", "bangkotan": "tua", "banpres": "bantuan presiden", "bansarkas": "bantuan sarana kesehatan", "bazis": "badan amal, zakat, infak, dan sedekah", "bcoz": "karena", "beb": "sayang", "bejibun": "banyak", "belom": "belum", "bener": "benar", "ber2": "berdua", "berdikari": "berdiri di atas kaki sendiri", "bet": "banget", "beti": "beda tipis", "beut": "banget", "bgd": "banget", "bgs": "bagus", "bhubu": "tidur", "bimbuluh": "bimbingan dan penyuluhan", "bisi": "kalau-kalau", "bkn": "bukan", "bl": "beli", "blg": "bilang", "blm": "belum", "bls": "balas", "bnchi": "benci", "bngung": "bingung", "bnyk": "banyak", "bohay": "badan aduhai", "bokep": "porno", "bokin": "pacar", "bole": "boleh", "bolot": "bodoh", "bonyok": "ayah ibu", "bpk": "bapak", "brb": "segera kembali", "brngkt": "berangkat", "brp": "berapa", "brur": "saudara laki-laki", "bsa": "bisa", "bsk": "besok", "bu_bu": "tidur", "bubarin": "bubarkan", "buber": "buka bersama", "bujubune": "luar biasa", "buser": "buru sergap", "bwhn": "bawahan", "byar": "bayar", "byr": "bayar", "c8": "chat", "cabut": "pergi", "caem": "cakep", "cama-cama": "sama-sama", "cangcut": "celana dalam", "cape": "capek", "caur": "jelek", "cekak": "tidak ada uang", "cekidot": "coba lihat", "cemplungin": "cemplungkan", "ceper": "pendek", "ceu": "kakak perempuan", "cewe": "cewek", "cibuk": "sibuk", "cin": "cinta", "ciye": "cie", "ckck": "ck", "clbk": "cinta lama bersemi kembali", "cmpr": "campur", "cnenk": "senang", "congor": "mulut", "cow": "cowok", "coz": "karena", "cpa": "siapa", "gokil": "gila", "gombal": "suka merayu", "gpl": "tidak pakai lama", "gpp": "tidak apa-apa", "gretong": "gratis", "gt": "begitu", "gtw": "tidak tahu", "gue": "saya", "guys": "teman-teman", "gws": "cepat sembuh", "haghaghag": "tertawa", "hakhak": "tertawa", "handak": "bahan peledak", "hansip": "pertahanan sipil", "hellow": "halo", "helow": "halo", "hi": "hai", "hlng": "hilang", "hnya": "hanya", "houm": "rumah", "hrs": "harus", "hubad": "hubungan angkatan darat", "hubla": "perhubungan laut", "huft": "mengeluh", "humas": "hubungan masyarakat", "idk": "saya tidak tahu", "ilfeel": "tidak suka", "imba": "jago sekali", "imoet": "imut", "info": "informasi", "itung": "hitung", "isengin": "bercanda", "iyala": "iya lah", "iyo": "iya", "jablay": "jarang dibelai", "jadul": "jaman dulu", "jancuk": "anjing", "jd": "jadi", "jdikan": "jadikan", "jg": "juga", "jgn": "jangan", "jijay": "jijik", "jkt": "jakarta", "jnj": "janji", "jth": "jatuh", "jurdil": "jujur adil", "jwb": "jawab", "ka": "kakak", "kabag": "kepala bagian", "kacian": "kasihan", "kadit": "kepala direktorat", "kaga": "tidak", "kaka": "kakak", "kamtib": "keamanan dan ketertiban", "kamuh": "kamu", "kamyu": "kamu", "kapt": "kapten", "kasat": "kepala satuan", "kasubbid": "kepala subbidang", "kau": "kamu", "kbar": "kabar", "kcian": "kasihan", "keburu": "terlanjur", "kedubes": "kedutaan besar", "kek": "seperti", "keknya": "kayaknya", "keliatan": "kelihatan", "keneh": "masih", "kepikiran": "terpikirkan", "kepo": "mau tahu urusan orang", "kere": "tidak punya uang", "kesian": "kasihan", "ketauan": "ketahuan", "keukeuh": "keras kepala", "khan": "kan", "kibus": "kaki busuk", "kk": "kakak", "klian": "kalian", "klo": "kalau", "kluarga": "keluarga", "klwrga": "keluarga", "kmari": "kemari", "kmpus": "kampus", "kn": "kan", "knl": "kenal", "knpa": "kenapa", "kog": "kok", "kompi": "komputer", "komtiong": "komunis Tiongkok", "konjen": "konsulat jenderal", "koq": "kok", "kpd": "kepada", "kptsan": "keputusan", "krik": "garing", "krn": "karena", "ktauan": "ketahuan", "ktny": "katanya", "kudu": "harus", "kuq": "kok", "ky": "seperti", "kykny": "kayanya", "laka": "kecelakaan", "lambreta": "lambat", "lansia": "lanjut usia", "lapas": "lembaga pemasyarakatan", "lbur": "libur", "lekong": "laki-laki", "lg": "lagi", "lgkp": "lengkap", "lht": "lihat", "linmas": "perlindungan masyarakat", "lmyan": "lumayan", "lngkp": "lengkap", "loch": "loh", "lol": "tertawa", "lom": "belum", "loupz": "cinta", "lowh": "kamu", "lu": "kamu", "luchu": "lucu", "luff": "cinta", "luph": "cinta", "lw": "kamu", "lwt": "lewat", "maaciw": "terima kasih", "mabes": "markas besar", "macem-macem": "macam-macam", "madesu": "masa depan suram", "maen": "main", "mahatma": "maju sehat bersama", "mak": "ibu", "makasih": "terima kasih", "malah": "bahkan", "malu2in": "memalukan", "mamz": "makan", "manies": "manis", "mantep": "mantap", "markus": "makelar kasus", "mba": "mbak", "mending": "lebih baik", "mgkn": "mungkin", "mhn": "mohon", "miker": "minuman keras", "milis": "mailing list", "mksd": "maksud", "mls": "malas", "mnt": "minta", "moge": "motor gede", "mokat": "mati", "mosok": "masa", "msh": "masih", "mskpn": "meskipun", "msng2": "masing-masing", "muahal": "mahal", "muker": "musyawarah kerja", "mumet": "pusing", "muna": "munafik", "munaslub": "musyawarah nasional luar biasa", "musda": "musyawarah daerah", "muup": "maaf", "muuv": "maaf", "nal": "kenal", "nangis": "menangis", "naon": "apa", "napol": "narapidana politik", "naq": "anak", "narsis": "bangga pada diri sendiri", "nax": "anak", "ndak": "tidak", "ndut": "gendut", "nekolim": "neokolonialisme", "nelfon": "menelepon", "ngabis2in": "menghabiskan", "ngakak": "tertawa", "ngambek": "marah", "ngampus": "pergi ke kampus", "ngantri": "mengantri", "ngapain": "sedang apa", "ngaruh": "berpengaruh", "ngawur": "berbicara sembarangan", "ngeceng": "kumpul bareng-bareng", "ngeh": "sadar", "ngekos": "tinggal di kos", "ngelamar": "melamar", "ngeliat": "melihat", "ngemeng": "bicara terus-terusan", "ngerti": "mengerti", "nggak": "tidak", "ngikut": "ikut", "nginep": "menginap", "ngisi": "mengisi", "ngmg": "bicara", "ngocol": "lucu", "ngomongin": "membicarakan", "ngumpul": "berkumpul", "ni": "ini", "nyasar": "tersesat", "nyariin": "mencari", "nyiapin": "mempersiapkan", "nyiram": "menyiram", "nyok": "ayo", "o/": "oleh", "ok": "ok", "priksa": "periksa", "pro": "profesional", "psn": "pesan", "psti": "pasti", "puanas": "panas", "qmo": "kamu", "qt": "kita", "rame": "ramai", "raskin": "rakyat miskin", "red": "redaksi", "reg": "register", "rejeki": "rezeki", "renstra": "rencana strategis", "reskrim": "reserse kriminal", "sni": "sini", "somse": "sombong sekali", "sorry": "maaf", "sosbud": "sosial-budaya", "sospol": "sosial-politik", "sowry": "maaf", "spd": "sepeda", "sprti": "seperti", "spy": "supaya", "stelah": "setelah", "subbag": "subbagian", "sumbangin": "sumbangkan", "sy": "saya", "syp": "siapa", "tabanas": "tabungan pembangunan nasional", "tar": "nanti", "taun": "tahun", "tawh": "tahu", "tdi": "tadi", "te2p": "tetap", "tekor": "rugi", "telkom": "telekomunikasi", "telp": "telepon", "temen2": "teman-teman", "tengok": "menjenguk", "terbitin": "terbitkan", "tgl": "tanggal", "thanks": "terima kasih", "thd": "terhadap", "thx": "terima kasih", "tipi": "TV", "tkg": "tukang", "tll": "terlalu", "tlpn": "telepon", "tman": "teman", "tmbh": "tambah", "tmn2": "teman-teman", "tmph": "tumpah", "tnda": "tanda", "tnh": "tanah", "togel": "toto gelap", "tp": "tapi", "tq": "terima kasih", "trgntg": "tergantung", "trims": "terima kasih", "cb": "coba", "y": "ya", "munfik": "munafik", "reklamuk": "reklamasi", "sma": "sama", "tren": "trend", "ngehe": "kesal", "mz": "mas", "analisise": "analisis", "sadaar": "sadar", "sept": "september", "nmenarik": "menarik", "zonk": "bodoh", "rights": "benar", "simiskin": "miskin", "ngumpet": "sembunyi", "hardcore": "keras", "akhirx": "akhirnya", "solve": "solusi", "watuk": "batuk", "ngebully": "intimidasi", "masy": "masyarakat", "still": "masih", "tauk": "tahu", "mbual": "bual", "tioghoa": "tionghoa", "ngentotin": "senggama", "kentot": "senggama", "faktakta": "fakta", "sohib": "teman", "rubahnn": "rubah", "trlalu": "terlalu", "nyela": "cela", "heters": "pembenci", "nyembah": "sembah", "most": "paling", "ikon": "lambang", "light": "terang", "pndukung": "pendukung", "setting": "atur", "seting": "akting", "next": "lanjut", "waspadalah": "waspada", "gantengsaya": "ganteng", "parte": "partai", "nyerang": "serang", "nipu": "tipu", "ktipu": "tipu", "jentelmen": "berani", "buangbuang": "buang", "tsangka": "tersangka", "kurng": "kurang", "ista": "nista", "less": "kurang", "koar": "teriak", "paranoid": "takut", "problem": "masalah", "tahi": "kotoran", "tirani": "tiran", "tilep": "tilap", "happy": "bahagia", "tak": "tidak", "penertiban": "tertib", "uasai": "kuasa", "mnolak": "tolak", "trending": "trend", "taik": "tahi", "wkwkkw": "tertawa", "ahokncc": "ahok", "istaa": "nista", "benarjujur": "jujur", "mgkin": "mungkin"}

def fix_slangwords(text):
    words = text.split()
    fixed_words = []

    for word in words:
        if word.lower() in slangwords:
            fixed_words.append(slangwords[word.lower()])
        else:
            fixed_words.append(word)

    fixed_text = ' '.join(fixed_words)
    return fixed_text

def tokenizing_text(text):
    return word_tokenize(text)

def filtering_text(tokens):
    stopwords_ind = set(stopwords.words('indonesian'))
    stopwords_eng = set(stopwords.words('english'))

    custom_stopwords = {
        'iya', 'yaa', 'gak', 'nya', 'na', 'sih', 'ku', 'di',
        'ga', 'ya', 'gaa', 'loh', 'kah', 'woi', 'woii', 'woy'
    }

    combined_stopwords = stopwords_ind.union(stopwords_eng).union(custom_stopwords)

    filtered_tokens = [word for word in tokens if word not in combined_stopwords]
    return filtered_tokens

def stemming_text(text):
    factory = StemmerFactory()
    stemmer = factory.create_stemmer()

    words = text.split()
    stemmed_words = [stemmer.stem(word) for word in words]
    return ' '.join(stemmed_words)

def to_sentence(tokens):
    return ' '.join(tokens)

In [8]:
# Apply the text cleaning functions to the DataFrame
def preprocess_text(df):
    df['cleaning_text'] = df['Review'].apply(cleaning_text)
    df['casefolding_text'] = df['cleaning_text'].apply(casefolding_text)
    df['fix_slangwords'] = df['casefolding_text'].apply(fix_slangwords)
    df['tokenizing_text'] = df['fix_slangwords'].apply(tokenizing_text)
    df['filtering_text'] = df['tokenizing_text'].apply(filtering_text)
    df['clean_text'] = df['filtering_text'].apply(to_sentence)
    return df

clean_df = preprocess_text(clean_df)

In [9]:
# Show summary information of the cleaned DataFrame
clean_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 58439 entries, 0 to 58499
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Review            58439 non-null  object
 1   cleaning_text     58439 non-null  object
 2   casefolding_text  58439 non-null  object
 3   fix_slangwords    58439 non-null  object
 4   tokenizing_text   58439 non-null  object
 5   filtering_text    58439 non-null  object
 6   clean_text        58439 non-null  object
dtypes: object(7)
memory usage: 3.6+ MB


# **Labeling**

In [10]:
# Prepare sentiment dictionary data
def load_lexicon(url):
    """
    Fetch CSV data from URL and return dictionary of word:score.
    """
    lexicon = {}
    response = requests.get(url)
    if response.status_code == 200:
        reader = csv.reader(StringIO(response.text), delimiter=',')
        for row in reader:
            lexicon[row[0]] = int(row[1])
    else:
        print(f"Failed to fetch lexicon data from {url}")
    return lexicon

lexicon_positive = load_lexicon('https://raw.githubusercontent.com/angelmetanosaa/dataset/main/lexicon_positive.csv')

lexicon_negative = load_lexicon('https://raw.githubusercontent.com/angelmetanosaa/dataset/main/lexicon_negative.csv')

In [11]:
# Calculate sentiment score and polarity from word list
def sentiment_analysis_lexicon_indonesia(text):
    """
    Calculate sentiment score and polarity for a list of words using positive and negative lexicons.
    """
    score = 0

    for word in text:
        if word in lexicon_positive:
            score += lexicon_positive[word]
        if word in lexicon_negative:
            score += lexicon_negative[word]

    if score > 0:
        polarity = 'positif'
    elif score < 0:
        polarity = 'negatif'
    else:
        polarity = 'netral'

    return score, polarity

In [12]:
# Apply sentiment analysis and add results to new columns
results = clean_df['filtering_text'].apply(sentiment_analysis_lexicon_indonesia)

results = list(zip(*results))

clean_df['polarity_score'] = results[0]
clean_df['polarity'] = results[1]

print(clean_df['polarity'].value_counts())

polarity
negatif    33556
positif    18669
netral      6214
Name: count, dtype: int64


# **Data Splitting**

In [13]:
# Data preparation and TF-IDF feature extraction for Machine Learning
clean_df['label'] = clean_df['polarity'].astype('category').cat.codes
y_ml = clean_df['label'].values
X_ml = clean_df['clean_text'].values

X_ml_train, X_ml_test, y_ml_train, y_ml_test = train_test_split(
    X_ml, y_ml, test_size=0.3, random_state=42
)

tfidf = TfidfVectorizer(max_features=5000)
X_ml_train_tfidf = tfidf.fit_transform(X_ml_train)
X_ml_test_tfidf = tfidf.transform(X_ml_test
)

In [14]:
# Data Preparation for Deep Learning
y_dl = pd.get_dummies(clean_df['polarity']).values
X_dl = clean_df['clean_text'].values

tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(X_dl)
sequences = tokenizer.texts_to_sequences(X_dl)
max_length = 100
padded_sequences = pad_sequences(sequences, maxlen=max_length)

X_dl_train, X_dl_test, y_dl_train, y_dl_test = train_test_split(
    padded_sequences, y_dl, test_size=0.2, random_state=42
)

# **Training**

In [15]:
# Logistic Regression with TF-IDF for sentiment analysis
lr_model = LogisticRegression(max_iter=1000)
lr_model.fit(X_ml_train_tfidf, y_ml_train)

y_train_pred_lr = lr_model.predict(X_ml_train_tfidf)
y_test_pred_lr = lr_model.predict(X_ml_test_tfidf)

print(f"LR - Training Accuracy: {accuracy_score(y_ml_train, y_train_pred_lr):.4f}")
print(f"LR - Testing Accuracy: {accuracy_score(y_ml_test, y_test_pred_lr):.4f}")

print("\nClassification Report - Testing Set:")
print(classification_report(y_ml_test, y_test_pred_lr, target_names=clean_df['polarity'].astype(str).unique()))

LR - Training Accuracy: 0.9241
LR - Testing Accuracy: 0.8919

Classification Report - Testing Set:
              precision    recall  f1-score   support

     negatif       0.90      0.96      0.93     10059
     positif       0.85      0.44      0.58      1854
      netral       0.89      0.91      0.90      5619

    accuracy                           0.89     17532
   macro avg       0.88      0.77      0.80     17532
weighted avg       0.89      0.89      0.88     17532



In [16]:
# Support Vector Machine (SVM) with TF-IDF for sentiment analysis
svm_model = SVC(kernel='linear')
svm_model.fit(X_ml_train_tfidf, y_ml_train)

y_train_pred_svm = svm_model.predict(X_ml_train_tfidf)
y_test_pred_svm = svm_model.predict(X_ml_test_tfidf)

print(f"SVM - Training Accuracy: {accuracy_score(y_ml_train, y_train_pred_svm):.4f}")
print(f"SVM - Testing Accuracy: {accuracy_score(y_ml_test, y_test_pred_svm):.4f}")

print("\nClassification Report - Testing Set:")
print(classification_report(y_ml_test, y_test_pred_svm, target_names=clean_df['polarity'].astype(str).unique()))

SVM - Training Accuracy: 0.9339
SVM - Testing Accuracy: 0.8978

Classification Report - Testing Set:
              precision    recall  f1-score   support

     negatif       0.91      0.96      0.94     10059
     positif       0.76      0.53      0.62      1854
      netral       0.90      0.91      0.90      5619

    accuracy                           0.90     17532
   macro avg       0.86      0.80      0.82     17532
weighted avg       0.89      0.90      0.89     17532



In [17]:
# LSTM model for sentiment analysis
model = Sequential([
    Embedding(input_dim=5000, output_dim=128, input_length=max_length),
    LSTM(128, return_sequences=True, dropout=0.2, recurrent_dropout=0.2),
    LSTM(64, dropout=0.2, recurrent_dropout=0.2),
    Dense(64, activation='relu'),
    Dropout(0.5),
    BatchNormalization(),
    Dense(3, activation='softmax')
])

model.compile(
    loss='categorical_crossentropy', 
    optimizer='adam', 
    metrics=['accuracy']
)

class AccuracyCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        if logs.get('accuracy') > 0.93 and logs.get('val_accuracy') > 0.93:
            print("\nAccuracy and val_accuracy > 93%. Training stopped.")
            self.model.stop_training = True

history = model.fit(
    X_dl_train, y_dl_train,
    epochs=50,
    batch_size=64,
    validation_data=(X_dl_test, y_dl_test),
    callbacks=[AccuracyCallback()]
)

Epoch 1/50
731/731 ━━━━━━━━━━━━━━━━━━━━ 154s 203ms/step - accuracy: 0.7029 - loss: 0.6867 - val_accuracy: 0.8837 - val_loss: 0.3246
Epoch 2/50
731/731 ━━━━━━━━━━━━━━━━━━━━ 136s 186ms/step - accuracy: 0.9022 - loss: 0.2815 - val_accuracy: 0.9114 - val_loss: 0.2514
Epoch 3/50
731/731 ━━━━━━━━━━━━━━━━━━━━ 160s 210ms/step - accuracy: 0.9249 - loss: 0.2158 - val_accuracy: 0.9167 - val_loss: 0.2288
Epoch 4/50
731/731 ━━━━━━━━━━━━━━━━━━━━ 151s 206ms/step - accuracy: 0.9361 - loss: 0.1847 - val_accuracy: 0.9212 - val_loss: 0.2221
Epoch 5/50
731/731 ━━━━━━━━━━━━━━━━━━━━ 174s 238ms/step - accuracy: 0.9440 - loss: 0.1577 - val_accuracy: 0.9250 - val_loss: 0.2284
Epoch 6/50
731/731 ━━━━━━━━━━━━━━━━━━━━ 181s 247ms/step - accuracy: 0.9521 - loss: 0.1413 - val_accuracy: 0.9271 - val_loss: 0.2250
Epoch 7/50
731/731 ━━━━━━━━━━━━━━━━━━━━ 186s 254ms/step - accuracy: 0.9558 - loss: 0.1330 - val_accuracy: 0.9281 - val_loss: 0.2476
Epoch 8/50
731/731 ━━━━━━━━━━━━━━━━━━━━ 0s 237ms/step - accuracy: 0.9608 - l

In [18]:
# Bidirectional LSTM model for sentiment analysis
model = Sequential([
    Embedding(input_dim=5000, output_dim=128, input_length=max_length),
    Bidirectional(LSTM(128, return_sequences=True, dropout=0.2, recurrent_dropout=0.2)),
    Bidirectional(LSTM(64, dropout=0.2, recurrent_dropout=0.2)),
    Dense(64, activation='relu'),
    Dropout(0.5),
    BatchNormalization(),
    Dense(3, activation='softmax')
])

model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

history = model.fit(
    X_dl_train, y_dl_train,
    epochs=50,
    batch_size=64,
    validation_data=(X_dl_test, y_dl_test),
    callbacks=[AccuracyCallback()]
)

Epoch 1/50
731/731 ━━━━━━━━━━━━━━━━━━━━ 796s 1s/step - accuracy: 0.7026 - loss: 0.6743 - val_accuracy: 0.8909 - val_loss: 0.3051
Epoch 2/50
731/731 ━━━━━━━━━━━━━━━━━━━━ 766s 1s/step - accuracy: 0.9040 - loss: 0.2739 - val_accuracy: 0.8990 - val_loss: 0.2778
Epoch 3/50
731/731 ━━━━━━━━━━━━━━━━━━━━ 842s 1s/step - accuracy: 0.9284 - loss: 0.2072 - val_accuracy: 0.9196 - val_loss: 0.2251
Epoch 4/50
731/731 ━━━━━━━━━━━━━━━━━━━━ 721s 986ms/step - accuracy: 0.9381 - loss: 0.1792 - val_accuracy: 0.9237 - val_loss: 0.2184
Epoch 5/50
731/731 ━━━━━━━━━━━━━━━━━━━━ 0s 938ms/step - accuracy: 0.9476 - loss: 0.1545
Accuracy and val_accuracy > 93%. Training stopped.
731/731 ━━━━━━━━━━━━━━━━━━━━ 715s 978ms/step - accuracy: 0.9476 - loss: 0.1546 - val_accuracy: 0.9316 - val_loss: 0.2110


In [19]:
# GRU model for sentiment analysis
model = Sequential([
    Embedding(input_dim=5000, output_dim=128, input_length=max_length),
    GRU(128, return_sequences=True, dropout=0.2, recurrent_dropout=0.2),
    GRU(64, dropout=0.2, recurrent_dropout=0.2),
    Dense(64, activation='relu'),
    Dropout(0.5),
    BatchNormalization(),
    Dense(3, activation='softmax')
])

model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

history = model.fit(
    X_dl_train, y_dl_train,
    epochs=50,
    batch_size=64,
    validation_data=(X_dl_test, y_dl_test),
    callbacks=[AccuracyCallback()]
)

Epoch 1/50
731/731 ━━━━━━━━━━━━━━━━━━━━ 120s 157ms/step - accuracy: 0.7021 - loss: 0.6939 - val_accuracy: 0.8890 - val_loss: 0.3079
Epoch 2/50
731/731 ━━━━━━━━━━━━━━━━━━━━ 102s 140ms/step - accuracy: 0.9018 - loss: 0.2848 - val_accuracy: 0.9066 - val_loss: 0.2543
Epoch 3/50
731/731 ━━━━━━━━━━━━━━━━━━━━ 106s 145ms/step - accuracy: 0.9261 - loss: 0.2177 - val_accuracy: 0.9122 - val_loss: 0.2483
Epoch 4/50
731/731 ━━━━━━━━━━━━━━━━━━━━ 117s 160ms/step - accuracy: 0.9347 - loss: 0.1879 - val_accuracy: 0.9129 - val_loss: 0.2542
Epoch 5/50
731/731 ━━━━━━━━━━━━━━━━━━━━ 123s 168ms/step - accuracy: 0.9419 - loss: 0.1656 - val_accuracy: 0.9168 - val_loss: 0.2473
Epoch 6/50
731/731 ━━━━━━━━━━━━━━━━━━━━ 131s 179ms/step - accuracy: 0.9502 - loss: 0.1470 - val_accuracy: 0.9291 - val_loss: 0.2163
Epoch 7/50
731/731 ━━━━━━━━━━━━━━━━━━━━ 128s 175ms/step - accuracy: 0.9529 - loss: 0.1344 - val_accuracy: 0.9273 - val_loss: 0.2258
Epoch 8/50
731/731 ━━━━━━━━━━━━━━━━━━━━ 135s 185ms/step - accuracy: 0.9597 -

In [20]:
# Bidirectional GRU model for sentiment analysis
model = Sequential([
    Embedding(input_dim=5000, output_dim=128, input_length=max_length),
    Bidirectional(GRU(128, return_sequences=True, dropout=0.2, recurrent_dropout=0.2)),
    Bidirectional(GRU(64, dropout=0.2, recurrent_dropout=0.2)),
    Dense(64, activation='relu'),
    Dropout(0.5),
    BatchNormalization(),
    Dense(3, activation='softmax')
])

model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

history = model.fit(
    X_dl_train, y_dl_train,
    epochs=50,
    batch_size=64,
    validation_data=(X_dl_test, y_dl_test),
    callbacks=[AccuracyCallback()]
)

Epoch 1/50
731/731 ━━━━━━━━━━━━━━━━━━━━ 591s 790ms/step - accuracy: 0.7024 - loss: 0.6875 - val_accuracy: 0.8981 - val_loss: 0.2902
Epoch 2/50
731/731 ━━━━━━━━━━━━━━━━━━━━ 597s 816ms/step - accuracy: 0.9095 - loss: 0.2654 - val_accuracy: 0.9134 - val_loss: 0.2377
Epoch 3/50
731/731 ━━━━━━━━━━━━━━━━━━━━ 595s 814ms/step - accuracy: 0.9292 - loss: 0.2057 - val_accuracy: 0.9085 - val_loss: 0.2845
Epoch 4/50
731/731 ━━━━━━━━━━━━━━━━━━━━ 496s 679ms/step - accuracy: 0.9360 - loss: 0.1841 - val_accuracy: 0.9286 - val_loss: 0.2122
Epoch 5/50
731/731 ━━━━━━━━━━━━━━━━━━━━ 452s 619ms/step - accuracy: 0.9457 - loss: 0.1566 - val_accuracy: 0.9293 - val_loss: 0.2219
Epoch 6/50
731/731 ━━━━━━━━━━━━━━━━━━━━ 595s 814ms/step - accuracy: 0.9505 - loss: 0.1466 - val_accuracy: 0.9288 - val_loss: 0.2191
Epoch 7/50
731/731 ━━━━━━━━━━━━━━━━━━━━ 686s 938ms/step - accuracy: 0.9568 - loss: 0.1344 - val_accuracy: 0.9239 - val_loss: 0.2554
Epoch 8/50
731/731 ━━━━━━━━━━━━━━━━━━━━ 0s 971ms/step - accuracy: 0.9589 - l

# **Inference**

In [25]:
# Test the model with a sample text
label_mapping = {0: 'negatif', 1: 'netral', 2: 'positif'}

def predict_sentiment(text):
    cleaned = cleaning_text(text)
    folded = casefolding_text(cleaned)
    fixed = fix_slangwords(folded)
    tokens = tokenizing_text(fixed)
    filtered = filtering_text(tokens)
    stemmed = stemming_text(' '.join(filtered))

    sequence = tokenizer.texts_to_sequences([stemmed])
    padded = pad_sequences(sequence, maxlen=max_length)

    prediction = model.predict(padded)
    predicted_label_index = prediction.argmax(axis=1)[0]
    sentiment = label_mapping[predicted_label_index]

    return sentiment

input_text = input("Enter a new sentence: ")
hasil = predict_sentiment(input_text)
print(f"Sentimen: {hasil}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 440ms/step
Sentimen: negatif
